In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [67]:
df=pd.read_csv("FutbolMatches.csv")
df=df.iloc[:,1:]
x=df.iloc[:,[6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,25,28,29,30,31,32,33,35,36,37,38,39,40,41,42,43,44,45,46]]
y=df.iloc[:,[34]]
df

,date,strong_team,weak_team,strong_score,weak_score,tournament,strong_overall,weak_overall,strong_attack,weak_attack,...,dif_midfield,dif_deffense,dif_bu_speed,dif_bu_passing,dif_cc_crossing,dif_cc_passing,dif_cc_shooting,dif_aggression,dif_preassure,dif_avg_age
0,2010-03-02,Brazil,Ireland,2,0,Friendly,84,75,83,72,...,7,9,0,-40,-10,0,0,-35,0,-0.73
1,2010-03-03,Denmark,Austria,1,2,Friendly,75,73,72,71,...,5,4,-20,-20,-15,-15,-15,-20,-5,3.03
2,2010-03-03,Croatia,Belgium,1,0,Friendly,78,76,77,73,...,-1,-1,5,-20,-25,5,-5,-15,0,2.04
3,2010-03-03,Spain,France,2,0,Friendly,85,83,83,82,...,1,0,-5,-10,-5,0,35,-15,5,-1.00
4,2010-03-03,Argentina,Germany,1,0,Friendly,83,82,83,81,...,2,-1,0,-32,23,11,0,-16,-28,1.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178,2018-03-27,Sweden,Romania,0,1,Friendly,76,73,77,75,...,4,4,-2,2,-10,14,19,10,26,-1.30
1179,2018-03-27,France,Russia,3,1,Friendly,84,76,83,78,...,10,5,-15,-19,16,-39,-11,-16,3,-2.35
1180,2018-03-27,Spain,Argentina,6,1,Friendly,86,83,84,87,...,4,5,-4,-12,-4,-9,-16,-4,-1,-0.48
1181,2018-03-27,USA,Paraguay,1,0,Friendly,75,74,76,76,...,1,1,2,1,-28,34,1,16,-11,-1.87


In [68]:
import warnings
warnings.filterwarnings('ignore')
def pre(df):
    teams = np.union1d(df["strong_team"].unique(),df["weak_team"].unique())
    tot_mat={}
    for team in teams:
        tot_mat[team]=0

    for team in teams:
        tot_mat[team]=len(df[(df["strong_team"]==team)|(df["weak_team"]==team)])
        
    for key in tot_mat.keys():
        if(tot_mat[key]<20):
            teams = teams[teams != key]
            
    df=df[(df["weak_team"].isin(teams))&(df["strong_team"].isin(teams))]
    
    df["strong_form"] = 0
    df["weak_form"] = 0
    df.reset_index(drop=True, inplace=True)
    
    df["strong_win"] = 0
    df["weak_win"] = 0
    for i in range(1082):
        if(df.iloc[i,:]["strong_score"] > df.iloc[i,:]["weak_score"]):
            df["strong_win"][i] = 1
            df["weak_win"][i] = -1
        elif(df.iloc[i,:]["strong_score"] < df.iloc[i,:]["weak_score"]):
            df["strong_win"][i] = -1
            df["weak_win"][i] = 1
    
    newdf = pd.DataFrame(index=range(0,100),columns=teams)
    for team in teams:
        for i in range(len(df[(df["strong_team"]==team)|(df["weak_team"]==team)])):
            if(df[(df["strong_team"]==team)|(df["weak_team"]==team)].iloc[i,1]==team):
                newdf[team][i] = df[(df["strong_team"]==team)|(df["weak_team"]==team)].iloc[i,:]["strong_win"]
            else:
                newdf[team][i] = df[(df["strong_team"]==team)|(df["weak_team"]==team)].iloc[i,:]["weak_win"]
                
    team_form = pd.DataFrame(index=range(0,94),columns=teams)
    x = 0
    pt = 0
    for team in teams:
        i = 1
        team_form[team][0] = newdf[team][0]
        while (newdf[team][i]!= np.nan)&(i<94):
            x = 0
            pt = 0
            if(i<=4):
                for r in range(i,-1,-1):
                    x=x+1
                    pt+=newdf[team][r]

            else:
                for r in range(i,i-5,-1):
                    x=x+1
                    pt+=newdf[team][r]


            team_form[team][i] = pt/x
            i+=1
    
    mat_count = {}
    for team in teams:
        mat_count[team] = 0

    df["strong_form"]=0
    df["weak_form"]=0
    df["strong_form"]=df["strong_form"].astype(float)
    df["weak_form"]=df["weak_form"].astype(float)
    
    for i in range(len(df)):
        df["strong_form"][i] = team_form[df["strong_team"][i]][mat_count[df["strong_team"][i]]]
        mat_count[df["strong_team"][i]]+=1
        df["weak_form"][i] = team_form[df["weak_team"][i]][mat_count[df["weak_team"][i]]]
        mat_count[df["weak_team"][i]]+=1
        
    del df["date"]
    del df["weak_win"]
    del df["strong_win"]
    del df['strong_cc_positioning']
    del df['weak_cc_positioning']
    del df['strong_bu_positioning']
    del df['weak_bu_positioning']
    
    return df.iloc[:,5:]

In [69]:
def train_test_transform(i):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=i)
    X_train=StandardScaler().fit_transform(X_train)
    X_test=StandardScaler().fit(X_train).transform(X_test)
    return X_train, X_test, y_train, y_test

In [77]:
pre(df).columns

Index(['strong_overall', 'weak_overall', 'strong_attack', 'weak_attack',
       'strong_midfield', 'weak_midfield', 'strong_deffense', 'weak_deffense',
       'strong_bu_speed', 'weak_bu_speed', 'strong_bu_passing',
       'weak_bu_passing', 'strong_cc_crossing', 'weak_cc_crossing',
       'strong_cc_passing', 'weak_cc_passing', 'strong_cc_shooting',
       'weak_cc_shooting', 'strong_aggression', 'weak_aggression',
       'strong_preassure', 'weak_preassure', 'strong_avg_age', 'weak_avg_age',
       'win', 'dif_overall', 'dif_attack', 'dif_midfield', 'dif_deffense',
       'dif_bu_speed', 'dif_bu_passing', 'dif_cc_crossing', 'dif_cc_passing',
       'dif_cc_shooting', 'dif_aggression', 'dif_preassure', 'dif_avg_age',
       'strong_form', 'weak_form'],
      dtype='object')

In [75]:
pre(df)["win"].index

RangeIndex(start=0, stop=1082, step=1)